
## dataset link
```
https://drive.google.com/file/d/1OKT4Q5fZ4rDBTY_bDwLSDg5-ZF1BcBeZ/view?usp=sharing
```



## Trying FL

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten,Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
import collections
from matplotlib import pyplot as plt

import cv2
import os
import shutil 
import math
import random
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define a function for federated averaging
def federated_averaging(client_model, centralized_model):
    client_weights = client_model.get_weights()
    central_weights = centralized_model.get_weights()
    for i in range(len(central_weights)):
        central_weights[i] = (central_weights[i] + client_weights[i])/2
    centralized_model.set_weights(central_weights)
    return centralized_model

In [3]:
#create clients
#clients = create_clients(X_train, y_train, num_clients=3, initial='client')

clients = []
clients.append('1')
clients.append('2')
clients.append('3')
#process and batch the training data for each client
clients_batched = dict()
for client_name in clients:
    # client_name format: clients_1
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(zoom_range = 0.2, shear_range = 0.2 , rescale = 1./255 , horizontal_flip=True)
    train_data = train_datagen.flow_from_directory(directory= "/kaggle/input/rakib-cervical/client test/client"+client_name, target_size=(66, 66), batch_size=100, class_mode = 'categorical')
    # clients_batched[client_name] = batch_data(data)
    clients_batched[client_name] = train_data
    
#process and batch the test set  
#test_data = train_datagen.flow_from_directory(directory= "/content/test"+client_name, target_size=(66, 66), batch_size=100, class_mode = 'categorical')
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_data = test_datagen.flow_from_directory(directory= "/kaggle/input/rakib-cervical/client test/test", target_size=(66, 66), batch_size=100, class_mode = 'categorical')

comms_round = 10

#create optimizer
lr = 0.013 
loss='categorical_crossentropy'
metrics = ['accuracy'] # f1_score, precision_score
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               ) 
# optimizer = tf.train.AdamOptimizer(learning_rate=lr, name='Adam-op').minimize(loss)

Found 1120 images belonging to 5 classes.
Found 1125 images belonging to 5 classes.
Found 1119 images belonging to 5 classes.
Found 685 images belonging to 5 classes.


In [4]:
# Model architecture: MLP
class SimpleMLP:
    @staticmethod
    def build():
        model = Sequential() 
        model.add(Conv2D(filters=16, kernel_size= (3,3), activation= 'relu', input_shape=(66,66,3)) )
        model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu' ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(rate=0.25))
        model.add(Flatten())
        model.add(Dense(units=64, activation='relu'))
        model.add(Dropout(rate=0.25))
        model.add(Dense(units=5, activation='sigmoid'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']  )
        
        return model

## Train local models

In [5]:
def train_single_model(global_model,client,image_list,label_list):
    data = list(zip(image_list, label_list)) #create_clients theke anchi single model er jonno
    random.shuffle(data) # create_clients
    
    smlp_local = SimpleMLP() 
    local_model = smlp_local.build(4356, 5)
    local_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    # step_0 : get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    # set local model weight to the weight of the global model
    local_model.set_weights(global_weights)
    
    # process and batch the training data for each client
    clients_batched[client] = batch_data(data)
    
    #fit local model with client's data
    tf.autograph.experimental.do_not_convert(local_model.fit(clients_batched[client], epochs=100, verbose=0))
    global_model = federated_averaging(local_model,global_model)
    
    K.clear_session()

    return global_model

In [6]:
# todo : 
    #     shob local model tader weight update gula send korbe and then ei list e append korbe (scaled_local_weight_list.append(scaled_weights))
         
    #     #clear session to free memory after each communication round
    #     K.clear_session()
        
    # #to get the average over all the local model, we simply take the sum of the scaled weights
    # average_weights = sum_scaled_weights(scaled_local_weight_list)

    # return average_weights

In [7]:
def train_local_models(current_client, global_weights,global_model):
     #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    #random.shuffle(client_names)

    smlp_temp = SimpleMLP()
    temp_model = smlp_global.build()
        
    #loop through each client and create new local model
    for client in client_names:
        print("Common Round no: ",end=" ")
        print(current_client,end="   ")
        print("Child no: ",end=" ")
        print(client)
        smlp_local = SimpleMLP()
        local_model = smlp_local.build()
        local_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        local_model.fit(clients_batched[client],steps_per_epoch=3, epochs=150, verbose=1)

        temp_model = federated_averaging(local_model,temp_model)

        K.clear_session()

    return temp_model

## Aggregation of Global Model


In [8]:
def aggregate(global_model):
    #commence global training loop
    for comm_round in range(comms_round):
        print("common round: ",end="")
        print(comm_round)
        # step_0 : get the global model's weights - will serve as the initial weights for all local models
        global_weights = global_model.get_weights()
        
        # step_1 : call the function for local models training
        global_model = train_local_models(comm_round, global_weights,global_model)
    return global_model

## Initialize the global Model

In [9]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build()
global_model = aggregate(global_model)
  

2023-01-31 13:30:07.807195: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-31 13:30:07.913792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-31 13:30:07.914884: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-31 13:30:07.917293: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

common round: 0
Common Round no:  0   Child no:  1


2023-01-31 13:30:11.151399: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/150


2023-01-31 13:30:12.774748: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


3/3 [==============================] - 8s 193ms/step - loss: 1.6080 - accuracy: 0.2233
Epoch 2/150
3/3 [==============================] - 1s 270ms/step - loss: 1.5962 - accuracy: 0.2367
Epoch 3/150
3/3 [==============================] - 1s 222ms/step - loss: 1.5820 - accuracy: 0.2591
Epoch 4/150
3/3 [==============================] - 1s 232ms/step - loss: 1.5718 - accuracy: 0.2400
Epoch 5/150
3/3 [==============================] - 1s 134ms/step - loss: 1.5609 - accuracy: 0.2545
Epoch 6/150
3/3 [==============================] - 1s 212ms/step - loss: 1.5299 - accuracy: 0.2600
Epoch 7/150
3/3 [==============================] - 1s 196ms/step - loss: 1.4974 - accuracy: 0.2600
Epoch 8/150
3/3 [==============================] - 1s 196ms/step - loss: 1.4602 - accuracy: 0.2867
Epoch 9/150
3/3 [==============================] - 1s 232ms/step - loss: 1.3905 - accuracy: 0.4545
Epoch 10/150
3/3 [==============================] - 1s 198ms/step - loss: 1.3712 - accuracy: 0.4167
Epoch 11/150
3/3 [===

In [10]:
global_model.save('cervical_cancer_best_model.h5')

In [11]:
global_model = load_model("./cervical_cancer_best_model.h5")

In [12]:
# Checking the Accuracy of the Model 
accuracy = global_model.evaluate_generator(generator= test_data)[1] 
print(f"The accuracy of your model is = {accuracy*100} %")

The accuracy of your model is = 82.1897804737091 %
